In [1]:
# Notebook Structure:
# Cell 1: Details regarding model version, initialization of required tools/packages
# Cell 2: Reading in data, defininig parameters and initializing result storing structures
# Cell 3: Function that optimizes DA Unit commitment
# Cell 4: Script that runs the DA optimization, looping over hours/days of year and scenarios 

In [1]:
##### Team Sunny Outlook #####
##### CEE 272R Project #####
# Members:
# Vignesh Venugopal
# Austin Park
# Yuchi Sun

####################################
############# Version 1 ############
####################################

####################################
######### Initialize tools #########
####################################

# Initialize JuMP to allow mathematical programming models
using JuMP, Gurobi


In [3]:
####################################
########### Import Data ############
####################################
# NOTE: We have defined the filenames of the true insolation and wind factors
# as "11".
# Read in hourly demand and rated capacity of generators
demandAtHour = readcsv("Demand - GW.csv")
costAndEmissions = readcsv("costAndEmissions.csv")
# Accurate insolation and wind factors to be used for real time dispatch calculations
insolationFactorAccurate = readcsv("SolarFactorPerturbed_11.csv")
windFactorAccurate = readcsv("WindFactorPerturbed_11.csv")

###############################
######### Define Sets #########
###############################

# Set of generator names
GENERATORS = ["solar-60","solar-61","solar-62","solar-63","solar-64",
"solar-65","solar-67","wind-60","wind-61","wind-62",
"nuclear-63","gas-62","coal-63","gas-peaker"]
nGENERATORS = length(GENERATORS)
nSolarGENERATORS = 7
nWindGENERATORS = 3
nRenewableGENERATORS = nSolarGENERATORS + nWindGENERATORS

###################################################
############ Define parameters and data ###########
###################################################

# Time constants
hoursInYear = 24*365 #8760 hours/year
daysInYear = 365 #NOTE: Change to 365 after debugging

# Create forecasting error levels
errorLevel=1:11
nErrorLevels = length(errorLevel)
# Create penetration levels
penetrationLevel = [7, 10, 20, 30, 40, 50]/100
nPenetrationLevels = size(penetrationLevel)[1]
# Create nameplate vector that reflects changes in renewable penetration level
nameplate_original = costAndEmissions[:,2]
penetrationMatrix = zeros(nGENERATORS,nPenetrationLevels) # Create penetration level matrix
shareSolarGrowth = 5/9
shareWindGrowth = 4/9

# Fill with initial values
for i=1:size(penetrationMatrix)[2]
    penetrationMatrix[:,i] = nameplate_original
end
# Fill penetration matrix with capacities corresponding to penetration level.
for i=2:nPenetrationLevels # Don't need to change first mix as it's the true penetration
    # Renewable capacity for given penetration level [GW]
    renewablesForGivenPenetration = penetrationLevel[i] * sum(penetrationMatrix[nRenewableGENERATORS+1:13,i])/(1-penetrationLevel[i])
    # Renewable capacity added to base case of 7% penetration [GW]
    newRenewables = renewablesForGivenPenetration - sum(penetrationMatrix[1:nRenewableGENERATORS,1])
    newSolar = shareSolarGrowth * newRenewables
    newWind = shareWindGrowth * newRenewables
    # Change solar capacity [GW]
    penetrationMatrix[1:nSolarGENERATORS,i] += newSolar / nSolarGENERATORS + penetrationMatrix[1:nSolarGENERATORS,1]
    # Change wind capacity [GW]
    penetrationMatrix[nSolarGENERATORS+1:nRenewableGENERATORS,i] += newWind / nWindGENERATORS + penetrationMatrix[nSolarGENERATORS+1:nRenewableGENERATORS,1]
end

########################################
### Define Result Storage Structure ####
########################################
# Initialize a dictionary to store results:
Results = zeros(nPenetrationLevels*nErrorLevels,13) # 13 is number of columns needed to store
# Results Columns:
# Penetration | Noise | % Solar | % Wind | % Nuclear | % Gas | % Coal | % Peaker | True Cost ....
# Emissions | % Renewable Curtailed | Abs Curtailed | # of hours when peaker operates
# Noise: [1,10] and true value is 11
# Penetration: [7,10,20,30,40,50] (% Capacity)


66×13 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                        ⋮            
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 

In [4]:

####################################
##### Function for DA market #######
####################################

function run_DA_Market(startHour, endHour, lastHourgeneratePower, lastHourdispatch)
    # Define the model name and solver. In this case, model name is "m"
    m = Model(solver=GurobiSolver())

    ####################################
    ######## Decision variables ########
    ####################################

    # Power to generate for every hour of the year, from every source [GW]
    @variable(m, generatePower[startHour:endHour,1:nGENERATORS] >= 0)
    # Ramping variable: Absolute change in power output (hour h minus hour h-1) [GW/hr]
    @variable(m, ramp[startHour:endHour,1:nDispatchable] >= 0)
    # Keep account of whether or not a plant started operation in hour 'h'.
    @variable(m, startup[startHour:endHour,1:nGENERATORS] >= 0)
    # Emissions generated as a result of power generation. Not constrained right now.
    @variable(m, carbonEmitted[startHour:endHour] >= 0)
    # Binary variable- 1 if Generator 'g' was operational in hour 'h',0 otherwise.
    @variable(m, dispatch[startHour:endHour,1:nGENERATORS], Bin)


    ######################################
    ######## Objective Functions #########
    ######################################

    @objective(m, Min, sum(sum(generatePower[h,g] * costAndEmissions[g,3] for g in 1:nGENERATORS) for h=startHour:endHour) + sum(sum(ramp[h,g]*rampCost[g] for g=1:nDispatchable) for h=startHour:endHour) + sum(sum(startup[h,g]*startupCost[g]  for g=1:nGENERATORS) for h=startHour:endHour))

    ######################################
    ############# Constraints ############
    ######################################

    # For every hour, supply equals demand
    @constraint(m, [h=startHour:endHour], sum(generatePower[h,g] for g in 1:nGENERATORS) == demandAtHour[h])

    # Supplied power has to fall between min 'ON' power and availability for that hour
    @constraint(m, [h=startHour:endHour, g=1:nGENERATORS], dispatch[h,g] * minGeneratePower[g] <= generatePower[h,g])
    @constraint(m, [h=startHour:endHour, g=1:nGENERATORS], generatePower[h,g] <= dispatch[h,g] * availableEnergy[h,g])

    # Keep account of how much CO2 is emitted in each hour
    @constraint(m, [h=startHour:endHour], carbonEmitted[h] == sum(generatePower[h,g] * costAndEmissions[g,6] for g=1:nGENERATORS))

    # Ramping constraint [decimal capacity / hr]
    # Note: Applies only to nuclear, non-peaking gas, and coal.
    # For 1st hour in each day
    @constraint(m, [h=startHour,g=1:nDispatchable-1], -rampLimit[g] <= (generatePower[h,g+nRenewableGENERATORS] - lastHourgeneratePower[g+nRenewableGENERATORS]) / nameplate[g+nRenewableGENERATORS])
    @constraint(m, [h=startHour,g=1:nDispatchable-1], (generatePower[h,g+nRenewableGENERATORS] - lastHourgeneratePower[g+nRenewableGENERATORS]) / nameplate[g+nRenewableGENERATORS] <= rampLimit[g])
    # For rest of hours in day
    @constraint(m, [h=startHour+1:endHour,g=1:nDispatchable-1], -rampLimit[g] <= (generatePower[h,g+nRenewableGENERATORS] - generatePower[h-1,g+nRenewableGENERATORS]) / nameplate[g+nRenewableGENERATORS])
    @constraint(m, [h=startHour+1:endHour,g=1:nDispatchable-1], (generatePower[h,g+nRenewableGENERATORS] - generatePower[h-1,g+nRenewableGENERATORS]) / nameplate[g+nRenewableGENERATORS] <= rampLimit[g])
    # Ramp will ultimately be the absolute change in power generated by each generator in subsequent time steps.
    # For 1st hour in each day
    @constraint(m, [h=startHour,g=1:nDispatchable], lastHourgeneratePower[g+nRenewableGENERATORS] - generatePower[h,g+nRenewableGENERATORS] <= ramp[h,g])
    @constraint(m, [h=startHour,g=1:nDispatchable], generatePower[h,g+nRenewableGENERATORS] - lastHourgeneratePower[g+nRenewableGENERATORS] <= ramp[h,g])
    # For rest of hours in day
    @constraint(m, [h=startHour+1:endHour,g=1:nDispatchable], generatePower[h-1,g+nRenewableGENERATORS] - generatePower[h,g+nRenewableGENERATORS] <= ramp[h,g])
    @constraint(m, [h=startHour+1:endHour,g=1:nDispatchable], generatePower[h,g+nRenewableGENERATORS] - generatePower[h-1,g+nRenewableGENERATORS] <= ramp[h,g])

    # Check if any generator started operating in hour 'h'
    # For 1st hour in each day
    @constraint(m, [h=startHour,g=1:nGENERATORS], startup[h,g] >= dispatch[h,g] - lastHourdispatch[g])
    # For rest of hours in day
    @constraint(m, [h=startHour+1:endHour,g=1:nGENERATORS], startup[h,g] >= dispatch[h,g] - dispatch[h-1,g])

    ####################################
    ########### Solve Model ############
    ####################################

    tic() # start timer
    solve(m)
    global evalTime = toc() # stop timer
    # Return decision variables that were solved for.
    return getvalue(generatePower[startHour:endHour,:]), getvalue(ramp[startHour:endHour,:]), getvalue(carbonEmitted[startHour:endHour]), getvalue(dispatch[startHour:endHour,:]), getvalue(startup[startHour:endHour,:]), getobjectivevalue(m), evalTime
end



run_DA_Market (generic function with 1 method)

In [5]:
########################################
#### Loop through penetration level ####
########################################
for pen in 1:nPenetrationLevels
    # Change penetration level
    global nameplate = penetrationMatrix[:,pen]

    # For real time dispatch using accurate capacity factors
    availableEnergyAccurate = zeros(hoursInYear,nGENERATORS)
    # Updating availableEnergyAccurate for solar generators
    for i=1:size(insolationFactorAccurate)[2]
        availableEnergyAccurate[1:hoursInYear,i] = nameplate[i] * insolationFactorAccurate[1:hoursInYear,i]
    end
    # Updating availableEnergyAccurate for wind generators
    for i=1:size(windFactorAccurate)[2]
        availableEnergyAccurate[1:hoursInYear,nSolarGENERATORS+i] = nameplate[nSolarGENERATORS+i] * windFactorAccurate[1:hoursInYear,i]
    end
    # Updating availableEnergy for remaining generators
    for i = nRenewableGENERATORS+1:nGENERATORS
        availableEnergyAccurate[1:hoursInYear,i] = nameplate[i]
    end

    ########################################
    #### Loop through error levels ####
    ########################################
    for err in 1:nErrorLevels
        ### Import perturbed data
        # Pertubed hourly capacity factors for renewables to be used for day ahead dispatch
        solarString = "SolarFactorPerturbed_"
        windString = "WindFactorPerturbed_"
        errStr = string(err)
        fileType = ".csv"
        insolationFactor = readcsv(string(solarString,errStr,fileType)) # Read in one of perturbed CF files
        windFactor = readcsv(string(windString,errStr,fileType))

        # Available energy (Max Rated Power * Cap factor) for each of generator (column) in each hour in a year (Row).
        # For day ahead dispatch using inaccurate capacity factors
        global availableEnergy = zeros(hoursInYear,nGENERATORS)
        # Updating availableEnergy for solar generators
        for i=1:size(insolationFactor)[2]
            availableEnergy[1:hoursInYear,i] = nameplate[i] * insolationFactor[1:hoursInYear,i]
        end
        # Updating availableEnergy for wind generators
        for i=1:size(windFactor)[2]
            availableEnergy[1:hoursInYear,nSolarGENERATORS+i] = nameplate[nSolarGENERATORS+i] * windFactor[1:hoursInYear,i]
        end
        # Updating availableEnergy for remaining generators
        for i = nRenewableGENERATORS+1:nGENERATORS
            availableEnergy[1:hoursInYear,i] = nameplate[i]
        end

        # Ramping limits [decimal capacity / hour] Note: decimal = %/100
        # Ramp limit is not defined for peaker coz we don't wanna constrain its ramping capability
        global rampLimit = [1, 0.60, 0.396]

        # Ramping costss [$/(GW/hr)]. This is the approximate cost for each generation
        # source to change its output by 1 GW over an hour. Dimensions: [4x1]
        global rampCost = [0.1, 0.615, 2.214, 0.615]*1000 # $/GW

        # Number of disptachable resources. Ramps are only calculated for dispatchable
        # resources: nuclear, gas, coal and gas-peaker.
        global nDispatchable = 4

        # Minimum output if 'ON' [GW]
        # Only for nuke, gas and coal, and NOT peaker
        global minGeneratePower = zeros(14,1)
        minGeneratePower[1:10] = 1e-8 # We want renewable generators to always disoatch when possible
        minGeneratePower[11:14] = [0.9, 0.25, 0.3, 1e-4] .* nameplate[11:14]

        # Startup cost [$]
        global startupCost = zeros(14,1)
        startupCost[11:14] = [3000,44000,61000,44000] .* nameplate[11:14]

        startHour = 1 # First hour of current loop iteration. Increments by 24.
        endHour = 24 # Last hour of current loop iteration. Increments by 24.
        # number of hours in day. Does not increment!
        hoursInDay = 24
        #
        # Initialize arrays to hold results
        YearOfGeneration = zeros(hoursInYear,nGENERATORS)
        YearOfRamp = zeros(hoursInYear,nDispatchable)
        YearOfCarbonEmitted = zeros(hoursInYear)
        YearOfDispatch = zeros(hoursInYear,nGENERATORS)
        YearOfStartup = zeros(hoursInYear,nGENERATORS)
        YearOfCost = zeros(daysInYear)
        YearOfCurtailment = zeros(hoursInYear) # Stores renewable curtailment (GWh)
        YearOfTrueCost = zeros(daysInYear) # True cost incurred after incorporating changes in real time ($)
        YearOfTrueCarbon = zeros(hoursInYear)

        # Initializing parameters that are passed on from previous to next optimization cycle.
        lastHourgeneratePower = zeros(1,nGENERATORS)
        lastHourdispatch = zeros(1,nGENERATORS)

        # Power Generation and dispatch is initialized to 0 at hour 0 of the year.
        lastHourgeneratePower = zeros(1,nGENERATORS)
        lastHourdispatch = zeros(1,nGENERATORS)

        # Loop over each day in the year
        for day=1:daysInYear

            ## Run DA market

            powerGeneration, Ramp, carbonEmission, Dispatch, Startup, Cost = run_DA_Market(startHour, endHour, lastHourgeneratePower, lastHourdispatch)
            # Store results in "YearOf" arrays
            YearOfGeneration[startHour:endHour,:] = powerGeneration
            YearOfRamp[startHour:endHour,:] = Ramp
            YearOfCarbonEmitted[startHour:endHour] = carbonEmission
            YearOfDispatch[startHour:endHour,:] = Dispatch
            YearOfStartup[startHour:endHour,:] = Startup
            YearOfCost[day] = Cost

            ## Run Real time market

            # Net Load for every hour from start to end of day of year in consideration (GWh)
            netLoad = demandAtHour[startHour:endHour] - sum(YearOfGeneration[startHour:endHour,nRenewableGENERATORS+1:end],2) - sum(availableEnergyAccurate[startHour:endHour,1:nRenewableGENERATORS],2)
            # Hours of year that have a net postive load to be satisfied by peakers
            hoursWithPositiveNetLoad = find(netLoad -> netLoad>0,netLoad) + (day-1) * 24
            # Hours of year that have a net negative load and thus observe renewable curtailment
            hoursWithNegativeNetLoad = find(netLoad -> netLoad<0,netLoad) + (day-1) * 24
            # Make note of renewable curtailments if any (GWh)
            YearOfCurtailment[hoursWithNegativeNetLoad] = -netLoad[hoursWithNegativeNetLoad - (day-1)*24]
            # Make note of extra peak power required (GWh)
            YearOfGeneration[hoursWithPositiveNetLoad,end] += netLoad[hoursWithPositiveNetLoad - (day-1)*24]
            # Make note of  hours when extra peak was required
            YearOfDispatch[hoursWithPositiveNetLoad,end] = 1
            #NOTE: We don't need to update YearOfGeneration[hoursWithNegativeNetLoad] because both solar
            # and wind are treated equally with same,negligible operating and startup costs. So it doesn't
            # matter if there was less wind but more solar as long as their summation meets or overexceeds expectations.
            # We would need to answer energy from which source is to be curtailed and to what extent
            # Make note of true renewable power generation on days it falls short
            YearOfGeneration[hoursWithPositiveNetLoad,1:nRenewableGENERATORS] = availableEnergyAccurate[hoursWithPositiveNetLoad,1:nRenewableGENERATORS]
            # Rectify hours when renewable generators are truly dispatched on days they fall short
            YearOfDispatch[hoursWithPositiveNetLoad,1:nRenewableGENERATORS] = 1*(availableEnergyAccurate[hoursWithPositiveNetLoad,1:nRenewableGENERATORS].>0)
            # Rectify YearOfRamp for peakers for 1st hour of day
            YearOfRamp[startHour,end] = abs(YearOfGeneration[startHour,end] - lastHourgeneratePower[end])
            # Rectify YearOfRamp for peakers for remaining hours of day
            YearOfRamp[startHour+1:endHour,end] = abs.(YearOfGeneration[startHour+1:endHour,end] - YearOfGeneration[startHour:endHour-1,end])
            # Rectifying YearOfStartup for renewable generators and peakers
            # 1st hour of day
            YearOfStartup[startHour,1:nRenewableGENERATORS] = max(YearOfDispatch[startHour,1:nRenewableGENERATORS] - lastHourdispatch[1:nRenewableGENERATORS],0)
            YearOfStartup[startHour,end] = max(YearOfDispatch[startHour,end] - lastHourdispatch[end],0)
            # Remaining hours of day
            YearOfStartup[startHour+1:endHour,1:nRenewableGENERATORS] = max.(YearOfDispatch[startHour+1:endHour,1:nRenewableGENERATORS] - YearOfDispatch[startHour:endHour-1,1:nRenewableGENERATORS],0)
            YearOfStartup[startHour+1:endHour,end] = max.(YearOfDispatch[startHour+1:endHour,end] - YearOfDispatch[startHour:endHour-1,end],0)
            # Calculating and storing true cost for the day as sum of operating, ramping and startup costs ($)
            YearOfTrueCost[day] = sum(YearOfGeneration[startHour:endHour,:]*costAndEmissions[:,3]) + sum(YearOfRamp[startHour:endHour,:]*rampCost) + sum(YearOfStartup[startHour:endHour,:]*startupCost)
            YearOfTrueCarbon[startHour:endHour] = YearOfGeneration[startHour:endHour,:] * costAndEmissions[:,6]

            # Set output in hour 24 as input for 1st hour of next day
            lastHourgeneratePower = YearOfGeneration[endHour,:]
            lastHourdispatch = YearOfDispatch[endHour,:]

            # Increment hours to next day
            startHour = day*hoursInDay + 1 # Gives 1st hour number for next loop iteration
            endHour = startHour + 23 # Gives  last hour number for next loop iteration
        end



        ##########################################
        ############ Printing Results ############
        ##########################################

        println("Days evaluated: ", floor(startHour/hoursInDay))
        println("Total operating cost \$: ", sum(YearOfTrueCost))
        println("Number of hours peaker is operated: ", sum(YearOfGeneration[:,14].>1e-2))
        println("Time to solve: ", evalTime)

        # Total power generation [GWh]
        powerGenerated = sum(YearOfGeneration)
        totalSolar = sum(YearOfGeneration[:,1:nSolarGENERATORS])
        totalWind = sum(YearOfGeneration[:,1+nSolarGENERATORS:nRenewableGENERATORS])
        totalNuclear = sum(YearOfGeneration[:,nRenewableGENERATORS+1])
        totalGas = sum(YearOfGeneration[:,nRenewableGENERATORS+2])
        totalCoal = sum(YearOfGeneration[:,nRenewableGENERATORS+3])
        totalGasPeaker = sum(YearOfGeneration[:,nGENERATORS])
        # Check that totalEnergy and demandAtHour have total energy
        totalEnergy = totalSolar + totalWind + totalNuclear + totalGas + totalCoal + totalGasPeaker

        # Positive accurate available energy
        renewAvailableEnergyAccurate = availableEnergyAccurate[:,1:nRenewableGENERATORS]
        posAvailableEnergyAccurate = renewAvailableEnergyAccurate[renewAvailableEnergyAccurate.>0]

        # supply = demand tolerance
        SDtolerance = 1e-4
        if totalEnergy - sum(demandAtHour) <= SDtolerance
            println("Energy generated by model is equal to demand over the whole year")
        else
            println("Supply does not equal demand over the whole year.")
        end

        # Energy mix [%]:
        genTypes = [totalSolar,totalWind,totalNuclear,totalGas,totalCoal,totalGasPeaker]
        energyMix = 100 * genTypes ./ totalEnergy
        println("Energy Mix: ", energyMix)

        # Total cost [$]:
        totalCost = sum(YearOfTrueCost)
        println("Total Cost: ", totalCost)

        # Store results from run in results
        Results[nErrorLevels * (pen - 1) + err, 1] = penetrationLevel[pen]
        Results[nErrorLevels * (pen - 1) + err, 2] = err
        Results[nErrorLevels * (pen - 1) + err, 3:8] = energyMix
        Results[nErrorLevels * (pen - 1) + err, 9] = sum(YearOfTrueCost)
        Results[nErrorLevels * (pen - 1) + err, 10] = sum(YearOfTrueCarbon)
        Results[nErrorLevels * (pen - 1) + err, 11] = sum(YearOfCurtailment)
        Results[nErrorLevels * (pen - 1) + err, 12] = sum(YearOfCurtailment)/sum(posAvailableEnergyAccurate)
        Results[nErrorLevels * (pen - 1) + err, 13] = sum(YearOfGeneration[:,14].>1e-2)
    end
end

# Write final results to csv
writecsv("Results_new.csv",Results)
print("You're a winner. This code works. Word.")


Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3518 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 4e+01]
Presolve removed 814 rows and 728 columns
Presolve time: 0.00s
Presolved: 578 rows, 400 columns, 1628 nonzeros
Variable types: 212 continuous, 188 integer (188 binary)
Found heuristic solution: objective 3.603475e+07
Found heuristic solution: objective 3.084232e+07
Found heuristic solution: objective 2.716857e+07

Root relaxation: objective 2.413877e+07, 172 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.413877e+07 2.4139e+07  0.00%     -    0s

Explored 0 nodes (172 simplex iterations) in 0.01 seconds


Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] max(::Array{Float64,1}, ::Int64) at .\deprecated.jl:57
 [3] macro expansion at .\In[5]:140 [inlined]
 [4] anonymous at .\<missing>:?
 [5] include_string(::String, ::String) at .\loading.jl:522
 [6] include_string(::Module, ::String, ::String) at C:\Users\vvign\.julia\v0.6\Compat\src\Compat.jl:71
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\vvign\.julia\v0.6\IJulia\src\execute_request.jl:154
 [8] (::Compat.#inner#17{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at C:\Users\vvign\.julia\v0.6\Compat\src\Compat.jl:385
 [9] eventloop(::ZMQ.Socket) at C:\Users\vvign\.julia\v0.6\IJulia\src\eventloop.jl:8
 [10] (::IJulia.##14#17)() at .\task.jl:335
while loading In[5], in expression starting on line 4


Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3517 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 4e+01]
Presolve removed 816 rows and 731 columns
Presolve time: 0.00s
Presolved: 576 rows, 397 columns, 1623 nonzeros
Variable types: 213 continuous, 184 integer (184 binary)
Found heuristic solution: objective 3.097893e+07
Found heuristic solution: objective 2.578435e+07
Found heuristic solution: objective 2.473065e+07

Root relaxation: objective 2.174449e+07, 141 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.174449e+07 2.1744e+07  0.00%     -    0s

Explored 0 nodes (141 simplex iterations) in 0.01 seconds


Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3523 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-02, 4e+01]
Presolve removed 806 rows and 727 columns
Presolve time: 0.00s
Presolved: 586 rows, 401 columns, 1654 nonzeros
Variable types: 215 continuous, 186 integer (186 binary)
Found heuristic solution: objective 3.089548e+07
Found heuristic solution: objective 2.569975e+07
Found heuristic solution: objective 2.198081e+07

Root relaxation: objective 2.181255e+07, 141 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.181255e+07 2.1813e+07  0.00%     -    0s

Explored 0 nodes (141 simplex iterations) in 0.01 seconds


Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3524 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 4e+01]
Presolve removed 816 rows and 728 columns
Presolve time: 0.00s
Presolved: 576 rows, 400 columns, 1614 nonzeros
Variable types: 214 continuous, 186 integer (186 binary)
Found heuristic solution: objective 2.565635e+07
Found heuristic solution: objective 2.332490e+07
Found heuristic solution: objective 2.299207e+07

Root relaxation: objective 1.996208e+07, 165 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1.996208e+07 1.9962e+07  0.00%     -    0s

Explored 0 nodes (165 simplex iterations) in 0.01 seconds


...

Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3520 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-02, 4e+01]
Presolve removed 811 rows and 729 columns
Presolve time: 0.00s
Presolved: 581 rows, 399 columns, 1633 nonzeros
Variable types: 215 continuous, 184 integer (184 binary)
Found heuristic solution: objective 2.958911e+07
Found heuristic solution: objective 2.439569e+07
Found heuristic solution: objective 2.084933e+07

Root relaxation: objective 2.073214e+07, 154 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.073214e+07 2.0732e+07  0.00%     -    0s

Explored 0 nodes (154 simplex iterations) in 0.01 seconds


Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3509 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-02, 4e+01]
Presolve removed 816 rows and 729 columns
Presolve time: 0.00s
Presolved: 576 rows, 399 columns, 1617 nonzeros
Variable types: 215 continuous, 184 integer (184 binary)
Found heuristic solution: objective 2.898640e+07
Found heuristic solution: objective 2.374894e+07
Found heuristic solution: objective 2.028533e+07

Root relaxation: objective 2.020160e+07, 153 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2.020160e+07 2.0202e+07  0.00%     -    0s

Explored 0 nodes (153 simplex iterations) in 0.01 seconds


Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3515 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 3e+01]
Presolve removed 821 rows and 728 columns
Presolve time: 0.00s
Presolved: 571 rows, 400 columns, 1596 nonzeros
Variable types: 214 continuous, 186 integer (186 binary)
Found heuristic solution: objective 2.428165e+07
Found heuristic solution: objective 2.175228e+07
Found heuristic solution: objective 2.173926e+07

Root relaxation: objective 1.879557e+07, 228 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1.879557e+07 1.8796e+07  0.00%     -    0s

Explored 0 nodes (228 simplex iterations) in 0.01 seconds


Academic license - for non-commercial use only
Optimize a model with 1392 rows, 1128 columns and 3519 nonzeros
Variable types: 792 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [1e-08, 9e+02]
  Objective range  [5e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 4e+01]
Presolve removed 817 rows and 729 columns
Presolve time: 0.00s
Presolved: 575 rows, 399 columns, 1613 nonzeros
Variable types: 215 continuous, 184 integer (184 binary)
Found heuristic solution: objective 2.852943e+07
Found heuristic solution: objective 2.334071e+07
Found heuristic solution: objective 1.993074e+07

Root relaxation: objective 1.980424e+07, 161 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1.980424e+07 1.9804e+07  0.00%     -    0s

Explored 0 nodes (161 simplex iterations) in 0.01 seconds


Excessive output truncated after 524931 bytes.